# Задача классификации

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.0; rv:14.0) Gecko/20100101 Firefox/14.0.1',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'ru-ru,ru;q=0.8,en-us;q=0.5,en;q=0.3',
    'Accept-Encoding':'gzip, deflate',
    'Connection':'keep-alive',
    'DNT':'1'
}
conn = sqlite3.connect('database.sqlite')

In [4]:
df = pd.read_csv('transfers.csv', sep=',', encoding='koi8-r')
del df['Unnamed: 0']
df.head()

,Date,Player,Profile,Type,Age,FromClub,FromType,FromCountry,ToClub,ToType,ToCountry,Fee,FeeNumber,TransferLink
0,1996-01-01,Chris Day,https://www.transfermarkt.co.uk/chris-day/prof...,Keeper,20.0,Spurs,Premier League,England,Crystal Palace,England,England,<U+00A3>315k,315000.0,https://www.transfermarkt.co.uk/jumplist/trans...
1,1996-01-01,Craig Armstrong,https://www.transfermarkt.co.uk/craig-armstron...,Left Midfield,20.0,Nottm Forest,Premier League,England,Bristol Rovers,England,England,Loan,NaN,https://www.transfermarkt.co.uk/jumplist/trans...
2,1996-01-01,Marquinhos Parana,https://www.transfermarkt.co.uk/marquinhos-par...,Defensive Midfield,18.0,Santa Cruz,Brazil,Brazil,Parana,Brazil,Brazil,Free transfer,0.0,https://www.transfermarkt.co.uk/jumplist/trans...
3,1996-01-01,Bosun Ayeni,https://www.transfermarkt.co.uk/bosun-ayeni/pr...,Defensive Midfield,17.0,V.I.P.,Nigeria,Nigeria,Shooting Stars,Nigeria,Nigeria,Free transfer,0.0,https://www.transfermarkt.co.uk/jumplist/trans...
4,1996-01-01,Chris Kanu,https://www.transfermarkt.co.uk/chris-kanu/pro...,Right-Back,16.0,Eagle Cement,Nigeria,Nigeria,AFC Ajax,Eredivisie,Netherlands,Free transfer,0.0,https://www.transfermarkt.co.uk/jumplist/trans...


In [18]:
ronaldo = df[(df.Player == 'Cristiano Ronaldo') & (df.Date == '2009-07-01')].iloc[0]
ronaldo

Date                                                   2009-07-01
Player                                          Cristiano Ronaldo
Profile         https://www.transfermarkt.co.uk/cristiano-rona...
Type                                               Centre-Forward
Age                                                            24
FromClub                                                  Man Utd
FromType                                           Premier League
FromCountry                                               England
ToClub                                                Real Madrid
ToType                                                     LaLiga
ToCountry                                                   Spain
Fee                                                <U+00A3>84.60m
FeeNumber                                                8.46e+07
TransferLink    https://www.transfermarkt.co.uk/jumplist/trans...
Name: 93823, dtype: object

### Сбор данных

In [24]:
url_ronaldo_info = ronaldo.Profile.replace('profil', 'leistungsdaten') + '/saison/' + str(ronaldo.Date[:4])
url_ronaldo_info

'https://www.transfermarkt.co.uk/cristiano-ronaldo/leistungsdaten/spieler/8198/saison/2009'

In [33]:
request = requests.get(url_ronaldo_info, headers=headers)
soup = BeautifulSoup(request.text, 'lxml')
# country
soup.find_all('span', attrs ={'class': 'dataValue'})[2].text.strip()

'Portugal'

In [40]:
# python code for scraping other player information

### Интеграция данных

In [56]:
ronaldo_before_sql = pd.read_sql_query('''
select *
from Player_Attributes
left join Player on Player_Attributes.player_fifa_api_id = Player.player_fifa_api_id
where Player.player_name = "Cristiano Ronaldo" and Player_Attributes.date like "%2009-08%" 
''', conn)
ronaldo_before_sql

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,gk_kicking,gk_positioning,gk_reflexes,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,33351,20801,30893,2009-08-30 00:00:00,89,94,right,high,low,86,...,71,21,21,1995,30893,Cristiano Ronaldo,20801,1985-02-05 00:00:00,185.42,176


In [57]:
# rating before transfer
ronaldo_before_sql['overall_rating']

0    89
Name: overall_rating, dtype: int64

In [61]:
ronaldo_after_sql = pd.read_sql_query('''
select *
from Player_Attributes
left join Player on Player_Attributes.player_fifa_api_id = Player.player_fifa_api_id
where Player.player_name = "Cristiano Ronaldo" and Player_Attributes.date like "%2010-08%" 
''', conn)
# rating after transfer
ronaldo_after_sql['overall_rating']

0    89
Name: overall_rating, dtype: int64

In [59]:
# data markup
ronaldo['IsSuccessTransfer'] = 0

In [60]:
ronaldo

Date                                                        2009-07-01
Player                                               Cristiano Ronaldo
Profile              https://www.transfermarkt.co.uk/cristiano-rona...
Type                                                    Centre-Forward
Age                                                                 24
FromClub                                                       Man Utd
FromType                                                Premier League
FromCountry                                                    England
ToClub                                                     Real Madrid
ToType                                                          LaLiga
ToCountry                                                        Spain
Fee                                                     <U+00A3>84.60m
FeeNumber                                                     8.46e+07
TransferLink         https://www.transfermarkt.co.uk/jumplist/trans...
IsSucc

In [ ]:
# python code (pd.merge) for join other player information